### This script uses the TXT files to create the NER prediction CSV files for all DaCy models.

In [ ]:
import re
import os
import glob
import pandas as pd

In [2]:
import dacy
for model in dacy.models():
    print(model)
    
nlp = dacy.load("da_dacy_large_trf-0.1.0")

da_dacy_small_tft-0.0.0
da_dacy_medium_tft-0.0.0
da_dacy_large_tft-0.0.0
da_dacy_small_trf-0.1.0
da_dacy_medium_trf-0.1.0
da_dacy_large_trf-0.1.0


C:\Users\Sarah\anaconda3\lib\site-packages\spacy\util.py:732: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Testing the functionalities of DaCy

In [6]:
doc = nlp("Opfordret af Deres Velbaarenhed til at erklære om jeg vedgaaer eller nægter at være Fader til et af Pigen Else Sophie Jensdatter Birkedalen født Drengebarn, som i Daaben den 25de October sidstleden er kaldet Hans Jacob, maa jeg herved ærbødigst meddele, at jeg ikke, uagtet Pigens Samqvem ogsaa med andre Mandspersoner paa den vedkommende Tid, bestemt tør fralægge mig bemeldte Paternitet, da jeg desværre med hende har pleiet legemlig Omgang, hvortil hendes fristende Adfærd og samtidige Tjeneste med mig hos Apotheker Reimann i lige Grad gav Anledning.Jeg er nu i 20de Aar gl:, eier aldeles Intet, uden nogle tarvelige Klæder, Skoetøi og Linned og skal om kort Tid forlade Grimstad Apothek, ved hvilket jeg som Lærling og altsaa uden nogen anden Løn, end Kosthold og de ovennævnte Fornødenheder, har opholdt mig siden Sommeren 1843. Min endnu levende Fader, til hvem jeg nødsages for det Første at begive mig, er en af de mindre Handelsmænd i Skien og befinder sig i høist maadelige Kaar.Til Vitterlighed:")

C:\Users\Sarah\anaconda3\lib\site-packages\spacy\pipeline\attributeruler.py:108: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True)


In [7]:
for entity in doc.ents:
   print(entity, ":", entity.label_)

Else Sophie Jensdatter Birkedalen : PER
Hans Jacob : PER
Apotheker : ORG
Reimann : PER
Grimstad Apothek : LOC
Skien : LOC


In [8]:
ents = [(e.label_, e.text) for e in doc.ents]
print(ents)

[('PER', 'Else Sophie Jensdatter Birkedalen'), ('PER', 'Hans Jacob'), ('ORG', 'Apotheker'), ('PER', 'Reimann'), ('LOC', 'Grimstad Apothek'), ('LOC', 'Skien')]


In [3]:
file_list = glob.glob(os.path.join(os.getcwd(), "C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\data_ibsen_work\\txt\\brev_p", "BREV*.txt"))

text = []
files = []

for file_path in file_list:
    with open(file_path, encoding = 'utf8') as f_input:
        t = f_input.read()
        text.append(t)
        file = (''.join([n for n in os.path.basename(file_path)]))
        file = re.sub('\.txt', '', file) 
        files.append(file)

In [4]:
ents = []

for doc in nlp.pipe(text, disable=["tagger", "parser", "attribute_ruler", "lemmatizer"]):
    ents.append(([(e.label_, e.text) for e in doc.ents]))

Token indices sequence length is longer than the specified maximum sequence length for this model (186 > 128). Running this sequence through the model will result in indexing errors


In [5]:
lists = {'files': files, 'text': text, 'dacy_small': ents}

df = pd.DataFrame.from_dict(lists, orient = 'index')
df = df.transpose()

In [6]:
df

,files,text,dacy_small
0,BREV_B1844-1871ht_18670308FH,,[]
1,BREV_B1844-1871ht_B18260306NTB,,[]
2,BREV_B1844-1871ht_B18440520PL,"Du maa virkelig undskylde, at jeg først nu bes...","[(ORG, Dit Brev), (PER, Mangel), (ORG, Brev), ..."
3,BREV_B1844-1871ht_B18441006HSte,"Tilgiv at jeg ikke før har besvaret Dit Brev, ...","[(PER, Sommer), (PER, Reimann), (MISC, Smaapig..."
4,BREV_B1844-1871ht_B18450801AWE,\nJomfru M: Wahl hilses venskabeligst fra,"[(PER, M), (PER, Wahl)]"
...,...,...,...
2444,BREV_B1890-1905ht_BudatNN_Hjerteligste,\nHjerteligste ønsker!Tør desværre ikke selv p...,[]
2445,BREV_B1890-1905ht_BudatNN_Med_udtrykket,\n\nMed udtrykket af min sympati for komitéens...,[]
2446,BREV_B1890-1905ht_BudatNN_Tallene,\nTallene må utvilsomt være skrevne af mig sel...,[]
2447,BREV_B1890-1905ht_BudatNN_Wenn_Sie,\n\n\n Wenn Sie keine andere Verwendung für Ih...,"[(LOC, Wenn Sie keine), (PER, andere Verwendun..."


In [7]:
path = (r"C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\data_ibsen_work\\csv\\")
df.to_csv(os.path.join(r'brev_dacy_small.csv'), encoding = 'utf-8', index = False)